<a href="https://colab.research.google.com/github/Yesn-t/TI_1_Practica3/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Amaro Lechuga Jashua Ricardo
**e-mail:** jashua.amaro3877@alumnos.udg.mx

## MODULES

In [17]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.figure_factory as ff

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [ ]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = pos_b - pos_a
    norm_ab = np.sqrt((vec_ab[0])**2 + (vec_ab[1])**2)

    vec_bc = pos_c - pos_b
    norm_bc = np.sqrt((vec_bc[0])**2 + (vec_bc[1])**2)

    dot_p = (pos_a[0] * pos_a[1]) + (pos_b[0] * pos_b[1]) + (pos_c[0] * pos_c[1])

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = 360 - cos_theta
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta


# BM Function
def bm_2d(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  Arguments:
    n_steps: Desc1
    speed: Desc2
    s_pos: Desc3
  Returns:
    BM_2d_df: return
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],
                          'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x,
                            'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df


# CRW Function
def CRW_2d(n_steps=1000, speed=6, s_pos=[0, 0],CRW_exponent=0.9):
  """
  Arguments:
    n_steps: Desc1
    speed: Desc2
    s_pos: Desc3
  Returns:
    CRW_2d_df: DataFrme con
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],
                          'y_pos':s_pos[1]}])

  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = wrapcauchy.rvs(CRW_exponent)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x,
                            'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])

    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

# Distancia Euclidiana
def euclidean_distance(points):
  distances = np.array([np.sqrt((points['x_pos'][i-1]-points['x_pos'][i])**2 + (points['y_pos'][i-1]-points['y_pos'][i])**2) for i in range(1, points.shape[0])])
  return distances

# Suma Cumulativa
def suma_cumulativa(distances):
  cumsum = np.zeros(len(distances))
  for i in range(1, len(cumsum)):
    cumsum[i] = cumsum[i-1] + distances[i]
  return cumsum



In [ ]:
a = np.array([1,-2])
b = np.array([4,5])
c = b-a
np.sqrt((c[0])**2 + (c[1])**2)

7.615773105863909

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = bm_2d(speed=3)

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = bm_2d(speed=6)

# Load existing trajectories to test your implementation
CRW_2d_df_6_9 = CRW_2d(speed=6, CRW_exponent=0.9)

In [ ]:
# Compute path length
## start - Add your code here
# dis_BM_3 = np.array([distance.euclidean(BM_2d_df_3.iloc[i-1],BM_2d_df_3.iloc[i]) for i in range(1,BM_2d_df_3.shape[0])])
# pl_BM_3 = np.cumsum(dis_BM_3)
dis_BM_3 = euclidean_distance(BM_2d_df_3)
pl_BM_3 = suma_cumulativa(dis_BM_3)

# dis_BM_6 = np.array([distance.euclidean(BM_2d_df_6.iloc[i-1],BM_2d_df_6.iloc[i]) for i in range(1,BM_2d_df_6.shape[0])])
# pl_BM_6 = np.cumsum(dis_BM_6)
dis_BM_6 = euclidean_distance(BM_2d_df_6)
pl_BM_6 = suma_cumulativa(dis_BM_6)

# dis_CRW_6_9 = np.array([distance.euclidean(CRW_2d_df_6_9.iloc[i-1],CRW_2d_df_6_9.iloc[i]) for i in range(1,CRW_2d_df_6_9.shape[0])])
# pl_CRW_6_9 = np.cumsum(dis_CRW_6_9)
dis_CRW_6_9 = euclidean_distance(CRW_2d_df_6_9)
pl_CRW_6_9 = suma_cumulativa(dis_CRW_6_9)
## end - Add your code here

In [ ]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3))+1,
    y = pl_BM_3,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_BM_3',
    showlegend=True
))
## end - Add your code here


# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_6))+1,
    y = pl_BM_6,
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    name='path_length_BM_6',
    showlegend=True
))
## end - Add your code here


# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_6_9))+1,
    y = pl_CRW_6_9,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_CRW_6_9',
    showlegend=True
))
## end - Add your code here


fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [ ]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))

# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here

    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
    ## start - Add your code here

    ## end - Add your code here

# Save metrics to Dataframe
## start - Add your code here

## end - Add your code here

# Write to csv
## start - Add your code here

## end - Add your code here

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here

## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here

## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
# CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')
CRW_2d_df_6_6 = CRW_2d(speed=6, CRW_exponent=0.6)

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
# CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')
CRW_2d_df_6_9 = CRW_2d(speed=6, CRW_exponent=0.9)

In [ ]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))
try_hist = []

# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6_6[1:-1].iterrows():
    ## start - Add your code here
    try_hist.append(turning_angle(CRW_2d_df_6_6.iloc[index-1], CRW_2d_df_6_6.iloc[index], CRW_2d_df_6_6.iloc[index+1]))
    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6_9[1:-1].iterrows():
    ## start - Add your code here
    a = True
    ## end - Add your code here


# Save to pandas DF
## start - Add your code here

## end - Add your code here


# Write to csv
## start - Add your code here

## end - Add your code here

<ipython-input-220-487b7ab43e33>:30: RuntimeWarning:

invalid value encountered in arccos



In [ ]:
Levy_M_df_1 = pd.read_csv('metrics/met_df_4.csv')
Levy_M_df_7 = pd.read_csv('metrics/met_df_5.csv')

In [ ]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(
    x = try_hist ))
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

x = np.linspace(0, 2*np.pi, 1000)
a_line = wrapcauchy.pdf(x, c=0.9)

fig_met_df_3.add_trace(go.Scatter(
    x = x,
    y = np.concatenate((a_line[int(len(a_line)/2)-1:-1], a_line[0:int(len(a_line)/2)])),
    mode='lines'))

x = np.linspace(0, 2*np.pi, 1000)
a_line = wrapcauchy.pdf(x, c=0.6)

fig_met_df_3.add_trace(go.Scatter(
    x = x,
    y = np.concatenate((a_line[int(len(a_line)/2)-1:-1], a_line[0:int(len(a_line)/2)])),
    mode='lines'))

## end - Add your code here

# fig_met_df_4.update(layout_xaxis_range = [0, 50])
# fig_met_df_4.update(layout_yaxis_range = [0, 0.4])
fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [ ]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

## start - Add your code here

## end - Add your code here


# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

## start - Add your code here

## end - Add your code here

In [13]:
Levy_M_df_1 = pd.read_csv('metrics/met_df_4.csv')
Levy_M_df_7 = pd.read_csv('metrics/met_df_5.csv')

In [66]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(
    x = Levy_M_df_1['SL_Levy_1'],
    histnorm='probability density',
    name='Histogram step-length Levy_2d_df_1',
    opacity=0.5,
    marker=dict(
        color='Red'),
    xbins=go.histogram.XBins(size=0.9)))
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(
    x = Levy_M_df_7['SL_Levy_7'],
    histnorm='probability density',
    name='Histogram step-length Levy_2d_df_1',
    opacity=0.5,
    marker=dict(
        color='Blue'),
    xbins=go.histogram.XBins(size=0.9)))
## end - Add your code here


# Add origin distributions
## start - Add your code here
x = np.linspace(0, 50, 1000)
a_line = levy_stable.pdf(x, alpha=1.0, beta=1.0, loc=3.0)

fig_met_df_4.add_trace(go.Scatter(
    x = x,
    y = a_line,
    name = 'Levy_1_1',
    marker=dict(
        color='Red'),
    mode='lines'))

a_line = levy_stable.pdf(x, alpha=0.7, beta=1.0, loc=3.)

fig_met_df_4.add_trace(go.Scatter(
    x = x,
    y = a_line,
    name = 'Levy_7_1',
    marker=dict(
        color='Blue'),
    mode='lines'))
## end - Add your code here

fig_met_df_4.update(layout_xaxis_range = [0, 50])
fig_met_df_4.update(layout_yaxis_range = [0, 0.4])
fig_met_df_4.show()